Ceci n'est pas un commentaire


### Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import kagglehub

In [ ]:
# Download latest version
path = kagglehub.dataset_download("mikhailma/test-dataset")

print("Path to dataset files:", path)

c:\Users\augus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 393M/393M [00:19<00:00, 20.9MB/s] 

Extracting files...


Path to dataset files: C:\Users\augus\.cache\kagglehub\datasets\mikhailma\test-dataset\versions\1


In [2]:
import os
import pandas as pd
from PIL import Image
import numpy as np


def lister_fichiers_recursivement(repertoire):
    data = {"Image": [], 
            "Label": []}
    #df = pd.DataFrame()
    for racine, repertoires, fichiers in os.walk(repertoire):
        for fichier in fichiers:

            sub_folder = fichier.split(" ")[0]
            if sub_folder == "Cross" : 
                sub_folder = "Crosswalk"
            if sub_folder == "Tlight" : 
                sub_folder = "Traffic Light"
            image = Image.open(repertoire+ "/" + sub_folder + "/"+ fichier).convert("RGB")
            image_array = np.array(image)
            data["Image"].append(image_array)
            data["Label"].append(sub_folder)

    return pd.DataFrame(data)
 
df = lister_fichiers_recursivement('./src/Google_Recaptcha_V2_Images_Dataset/images')

In [3]:
df.head(5)

,Image,Label
0,"[[[117, 114, 104], [114, 110, 101], [112, 109,...",Bicycle
1,"[[[54, 75, 67], [58, 76, 69], [66, 77, 71], [6...",Bicycle
2,"[[[38, 33, 27], [37, 34, 28], [37, 34, 29], [3...",Bicycle
3,"[[[49, 49, 51], [55, 56, 58], [72, 73, 75], [9...",Bicycle
4,"[[[135, 128, 125], [143, 130, 129], [153, 135,...",Bicycle


In [ ]:
from sklearn.model_selection import train_test_split

#train_images, train_labels, test_images, test_labels = train_test_split(df, )

In [ ]:
def build_model(dense_neurons):
    model = tf.keras.Sequential([
        layers.InputLayer((28,28,1)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(dense_neurons, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
model = build_model(dense_neurons=64)
BAT = 64
    # Train the model
history = model.fit(
    train_images,train_labels,
    batch_size=BAT,
    epochs=5,
    verbose=2
)
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)

How to load images with tensorflow  : https://www.tensorflow.org/api_docs/python/tf/keras/utils/load_img

In [ ]:
tf.keras.utils.load_img(
    path,
    color_mode='rgb',
    target_size=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)